In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 1, 0])              # Boundary condition function coefficients
dtype = torch.float                  # Tensor data type
supervised_samples = {n for n in range(16)}
unsupervised_samples = {n for n in range(16, 128)}

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
#Specify right hand side and stiffness matrix
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
K = PoissonFEM.StiffnessMatrix(mesh)
rhs.set_rhs_stencil(mesh, K)

In [5]:
trainingData = dta.StokesData(supervised_samples, unsupervised_samples)
trainingData.read_data()
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs, trainingData.output_resolution**2)

In [7]:
model = gs.GenerativeSurrogate(rom, trainingData)

In [ ]:
model.fit(n_steps=200)

step =  0
z step =  1.2995219230651855 s
thetaf step =  1.6900863647460938 s
lambdac step =  1.1811614036560059 s
uc =  [0.         2.84890628 2.9277597  3.4022875  3.74802107 2.77388327
 3.00518073 3.28656931 3.75211338 4.08557    3.06453912 3.37264975
 3.7258671  4.08121556 4.66372209 3.46933652 3.77438914 4.13853849
 4.57964492 4.92819111 3.7713987  4.06747998 4.45726182 4.81491811
 6.00169285]
thetac step =  1.1403789520263672 s
step =  1
z step =  1.3689041137695312 s
thetaf step =  1.5899779796600342 s
lambdac step =  0.7691495418548584 s
uc =  [0.         2.33884499 2.40557627 2.5418009  3.10898183 2.57594097
 2.56603791 2.56822845 3.11854921 3.65702798 2.598663   2.68888029
 3.02346561 3.42958038 4.13336082 2.75377823 2.988955   3.25638036
 3.79001189 4.21416951 2.99486688 3.24777931 3.59574265 3.96612785
 4.30532205]
thetac step =  0.6903722286224365 s
step =  2
z step =  1.257169246673584 s
thetaf step =  1.6295537948608398 s
lambdac step =  0.937105655670166 s
uc =  [0.     

thetac step =  1.0963881015777588 s
step =  18
z step =  1.3110427856445312 s
thetaf step =  1.6406493186950684 s
lambdac step =  1.1953537464141846 s
uc =  [ 0.         27.33305092 27.4211368  27.69076491 28.020303   27.53337556
 27.5440852  27.64295412 28.03742318 28.31560075 27.60936873 27.67243673
 27.9121627  28.25528666 28.62221862 27.67264719 27.78733231 28.02749153
 28.60933627 28.95031868 27.82392369 28.04838295 28.38738727 28.93838476
 29.44543023]
thetac step =  0.6185290813446045 s
step =  19
z step =  1.3138442039489746 s
thetaf step =  1.5153899192810059 s
lambdac step =  0.9989278316497803 s
uc =  [ 0.         19.75665074 19.75812574 19.65435453 19.92177533 19.80454592
 19.76382523 19.76197084 19.91468302 20.20338078 19.81862857 19.94376535
 19.96920087 20.00436221 20.14767821 19.92970727 19.965822   19.98159147
 20.03937607 20.06802819 19.96290641 19.99027438 20.02934923 20.05180318
 20.06751163]
thetac step =  0.9591083526611328 s
step =  20
z step =  1.337040901184082

z step =  1.3411219120025635 s
thetaf step =  1.5922293663024902 s
lambdac step =  1.2329716682434082 s
uc =  [0.         0.08732873 0.13377355 0.25407247 0.2584233  0.12917969
 0.16785028 0.21814546 0.25839814 0.26282443 0.20412827 0.22671713
 0.2603359  0.31300614 0.35380644 0.23503073 0.25847027 0.28767327
 0.33519317 0.37105546 0.25803276 0.28015976 0.30579078 0.37009986
 0.94906186]
thetac step =  1.0158259868621826 s
step =  37
z step =  1.2248687744140625 s
thetaf step =  1.5257854461669922 s
lambdac step =  0.651940107345581 s
uc =  [ 0.         21.53320255 21.54380603 21.98774987 22.38325715 21.52583695
 21.55273166 21.68431587 22.41265652 22.71996569 21.53633053 21.60555562
 22.28848951 22.69624814 23.1408789  21.68770945 22.27571909 22.63755343
 23.31059803 23.58581816 22.26096683 22.80471968 23.33416044 23.76346038
 24.15056135]
thetac step =  0.6623873710632324 s
step =  38
z step =  1.3038668632507324 s
thetaf step =  1.5931479930877686 s
lambdac step =  0.715863466262817

thetac step =  0.9897861480712891 s
step =  55
z step =  1.2654690742492676 s
thetaf step =  1.5833470821380615 s
lambdac step =  1.0526220798492432 s
uc =  [0.         0.09645617 0.1276605  0.15406299 0.23276131 0.14015777
 0.15991425 0.17382585 0.23252044 0.31194137 0.18925975 0.21719507
 0.24319933 0.28803928 0.34299167 0.22147775 0.24284974 0.27018307
 0.31093684 0.35675784 0.24289856 0.264417   0.29424709 0.32558161
 0.37264089]
thetac step =  0.9910337924957275 s
step =  56
z step =  1.3136379718780518 s


In [9]:
model.tauc

tensor([7.0532e-07, 1.6970e-05, 1.0261e-07, 1.6349e-05, 2.2444e-05, 5.9189e-07,
        7.5277e-06, 8.2528e-07, 6.6497e-07, 7.4825e-06, 2.4146e-07, 5.9673e-08,
        1.2487e-06, 7.5052e-07, 7.8022e-07, 2.7693e-04])

In [10]:
model.log_lambda_c_mean

AttributeError: 'GenerativeSurrogate' object has no attribute 'log_lambda_c_mean'

In [ ]:
rom.solution.array

In [ ]:
y = torch.tensor([1, 2, 3])

In [ ]:
y  + 1.2

In [ ]:
model.tauc

In [ ]:
# model.save()

In [ ]:
model2 = gs.GenerativeSurrogate()

In [ ]:
model2.load()

In [ ]:
model2.data.__init__(supervised_samples, unsupervised_samples)

In [ ]:
model2.data.read_data()

In [ ]:
model2.fit(n_steps=2)

In [ ]:
s = torch.multinomial(torch.ones(model.data.n_unsupervised_samples), model.batch_size_z)

In [ ]:
s

In [ ]:
torch.tensor(1.0)/(1 + s)